In [52]:
import numpy as np
import pandas as pd
import time
from scipy.linalg import eigh
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LinearRegression
import statsmodels
import statsmodels.api as sm

import cir
from cir import CIR
from importlib import reload
reload(cir)


data = pd.read_table('Retinol.txt', header=None, delim_whitespace=True)
data = data.iloc[:, :]

# foreground data 
fg = data.iloc[:, :12].dropna()

# foreground response (continuous)
Y = data.iloc[:, 12]
n, p = fg.shape  # foreground sample size
X = fg - np.mean(fg, axis=0)
X = X.values

# foreground slices 
L = 10
partition = np.linspace(min(Y), max(Y), L)
labels = np.zeros(n)
for i in range(n):
    labels[i] = max(np.where(Y.iloc[i] >= partition)[0]) + 1

# background data 
bg = data.iloc[:, :12].dropna()
m, p = bg.shape     # background sample size

# background data
Yt = data.iloc[:, 13]
Lt = 10
partitiont = np.linspace(min(Yt), max(Yt), Lt)
labelst = np.zeros(m)
for i in range(m):
    labelst[i] = max(np.where(Yt.iloc[i] >= partitiont)[0]) + 1

d = 2
alpha = 0.1


V_CIR = CIR(fg,labels,bg,labelst,alpha,d)
vvt = V_CIR @ V_CIR.T

X_CIR = X @ V_CIR
# print(V_CIR[:10, :])

print("Training Linear Model")
model_CIR_LM = LinearRegression().fit(X_CIR, Y)
print("coefficients: ", model_CIR_LM.coef_)
print("intercept: ", model_CIR_LM.intercept_)
print("R-squared: ", model_CIR_LM.score(X_CIR, Y))



m1 = [[0.532599608105052,   0.004620083626876,  -0.000320112347451,  -0.006601571813602,  -0.000499345812859,   0.295110986421280,  -0.005625130202443],
   [0.004620083626876,   0.926065049338340,  -0.000978627564536,   0.221573065829260,   0.004981206774528,   0.000999866145938,   0.078676661438859],
  [-0.000320112347451,  -0.000978627564536,   0.000001220756928,  -0.000229587592840,  -0.000004953664970,  -0.000175728731562,  -0.000079580448973],
  [-0.006601571813602,   0.221573065829260,  -0.000229587592840,   0.053125756655810,   0.001199404003959,  -0.004031287483785,   0.018911493104884],
  [-0.000499345812859,   0.004981206774528,  -0.000004953664970,   0.001199404003959,   0.000027309337619,  -0.000285084491660,   0.000429116469334],
  [0.295110986421280 ,  0.000999866145938 , -0.000175728731562 , -0.004031287483785 , -0.000285084491660,   0.163522264826441,  -0.003249489741347],
  [-0.005625130202443,   0.078676661438859,  -0.000079580448973,   0.018911493104884,   0.000429116469334,  -0.003249489741347,   0.006752208134958],
   [0.010839350364305,  -0.064090742373393,   0.000061123442157,  -0.015496025558275,  -0.000355720894400,   0.006114167969579,  -0.005571111050421],
  [-0.010753209254731,   0.094933481291968,  -0.000093676693721,   0.022876530750223,   0.000521687418020,  -0.006118397967805,   0.008192214667748],
  [-0.282957166594681,  -0.001048554700505,   0.000168586240285,   0.003843755046652,   0.000272859775825,  -0.156787629668804,   0.003108023113917],
   [0.015239189508293,  -0.001950955622589,  -0.000006964089265,  -0.000687460621746,  -0.000025502974935,   0.008447474162690,  -0.000338048665003],
  [-0.284990767155745,  -0.003305043920851,   0.000172167822695,   0.003333126351607,   0.000262712879579,  -0.157910666507893,   0.002939166922469]]



VVt_s = vvt[:12, :7]
print(VVt_s)
norm = np.linalg.norm(VVt_s-m1, ord='fro')
print("This is the norm of the difference")
print(norm)



# # tuning parameter
# alpha = 1.5
# d = 2

# # CIR 
# print("CIR......")
# V_CIR = CIR(fg,Y,bg,Yt,alpha,d)
# X_CIR = X @ V_CIR

# # colors = 228/255
# markers = 'o'
# colors = [[228/255, 26/255, 28/255],
#           [55/255, 126/255, 184/255],
#           [77/255, 175/255, 74/255],
#           [152/255, 78/255, 163/255],
#           [255/255, 127/255, 0/255],
#           [255/255, 255/255, 51/255],
#           [166/255, 86/255, 40/255],
#           [247/255, 129/255, 191/255],
#           [153/255, 153/255, 153/255]]


# markers = ['o', 's', 'p', 'o', 's', 'p', 'o', 's', 'p']

# for l in range(L):
#     idx = np.where(Y == labels[l])[0]
#     X_curr = X_CIR[idx, :]
#     plt.scatter(X_curr[:, 0], X_curr[:, 1], s=100, c=colors, marker=markers)
# # plt.hold(False)
# plt.title('CIR', fontsize=32)
# plt.xticks(fontsize=22)
# plt.yticks(fontsize=22)
# plt.show()

/Users/lianama/CIR/contrastive-inverse-regression/testing/cir.py:384: LinAlgWarning: Ill-conditioned matrix (rcond=3.64898e-19): result may not be accurate.
  aa = solve(eye2k + (alpha * tau) * VU, VX)
/Users/lianama/CIR/contrastive-inverse-regression/testing/cir.py:384: LinAlgWarning: Ill-conditioned matrix (rcond=3.64894e-19): result may not be accurate.
  aa = solve(eye2k + (alpha * tau) * VU, VX)
/Users/lianama/CIR/contrastive-inverse-regression/testing/cir.py:384: LinAlgWarning: Ill-conditioned matrix (rcond=3.64804e-19): result may not be accurate.
  aa = solve(eye2k + (alpha * tau) * VU, VX)
/Users/lianama/CIR/contrastive-inverse-regression/testing/cir.py:384: LinAlgWarning: Ill-conditioned matrix (rcond=3.6296e-19): result may not be accurate.
  aa = solve(eye2k + (alpha * tau) * VU, VX)
/Users/lianama/CIR/contrastive-inverse-regression/testing/cir.py:384: LinAlgWarning: Ill-conditioned matrix (rcond=3.55851e-19): result may not be accurate.
  aa = solve(eye2k + (alpha * tau) *

---------------------------------------------------

Results for Scaled Gradient Projection Method 

---------------------------------------------------

   Obj. function = 3.891533e+05

   Gradient norm = 1.229389e+01 

   ||X^T*X-I||_F = 4.97e-16

   Iteration number = 3000

   Cpu time (secs) = 5.9020

   Number of evaluation(Obj. func) = 4123

Training Linear Model
coefficients:  [ 0.122791602837945 10.507565601567029]
intercept:  189.8920634920635
R-squared:  0.11909860569817732
[[ 0.464681916533837  0.01165588773159  -0.000580249559588
  -0.010579890501132 -0.000872963064333  0.286681505968123
  -0.005675816524388]
 [ 0.01165588773159   0.791787840908798 -0.002603007649836
   0.37084947439795   0.007108920179656  0.006310329557888
   0.089000357272796]
 [-0.000580249559588 -0.002603007649836  0.000009189659409
  -0.00120045762786  -0.000022230067966 -0.000355099884764
  -0.000284438899952]
 [-0.010579890501132  0.37084947439795  -0.00120045762786
   0.174248494940151  0.003363359